In [1]:
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify
import pickle
import yaml
from yaml.loader import SafeLoader
import utils.preprocess as p
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder

import json

In [2]:
def get_params(resources_path = 'models/resources.yaml'):
    with open(resources_path, 'r') as f:
        resources = yaml.load(f, Loader=SafeLoader)
    return resources.get('preprocess_params')

In [3]:
def get_model(resources_path = 'models/resources.yaml'):
    with open(resources_path, 'r') as f:
        resources = yaml.load(f, Loader=SafeLoader)
        path = resources.get('prod_model_path')
    with open('models/' + path, 'rb') as infile:
        model = pickle.load(infile)
    return model

In [4]:
def get_pipeline(resources_path = 'models/resources.yaml'):
    with open(resources_path, 'r') as f:
        resources = yaml.load(f, Loader=SafeLoader)
        path = resources.get('prod_pipeline_path')
    with open('models/' + path, 'rb') as infile:
        pipeline = pickle.load(infile)
    return pipeline

In [5]:
def process_input_data(data):
    preprocess_f_list = [p.create_country_features,
                         p.create_source_features, p.create_platform_features, p.create_device_features,
                         p.create_event_1_flag, p.create_event_2_flag, p.drop_metadata_columns
    ]
    data = p.apply_pipeline(preprocess_f_list, data, params)
    return pipeline.transform(data)

In [6]:
model = get_model()
pipeline = get_pipeline()
params = get_params()

In [11]:
def predict():
    data = json.loads(
    '''{
    "user_id": ["abc"],
    "country": ["ar"],
    "source": ["Organic"],
    "platform": ["ios"],
    "device_family": ["Samsung Galaxy Tab"],
    "event_1": [10],
    "event_2": [1]
    }''')
    
    # Transformación de los datos de entrada
    data = pd.DataFrame.from_dict(data)
    data = process_input_data(data)   
    
    print(data)

    # Predicción
    prediction = model.predict(data)[0]
    if prediction <= 0:
        prediction = 0
        
    print(prediction)

    return

In [12]:
predict()

[[ 1.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   1.          0.          1.          1.         -0.38948921 -0.18028864
   0.99503719  0.09950372]]
0.026006326964222188
